<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.3
# *Google BigQuery and Gemini API*

## Introduction

The Google BigQuery UI provides access to Google's extensive collection of public data sets via an SQL-based query engine.

The BigQuery API provides programmatic access to the data sets.

The Google Gemini API provides programmatic access to Google's Generative AI models.

Each of these is explored in this lab.

## BigQuery Web UI

The Google BigQuery UI can be used to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

Work through the Quickstart at https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui.

You will need to set up a Google Cloud Platform account if you don't already have one. (This should not cost anything during the trial period unless you perform a large amount of querying. Afterwards, costs are based on actual resource usage, but most offerings have a free tier.)

## BigQuery API

- Open Google Cloud Console (https://console.cloud.google.com/home/) and select to create a project. A project is required to enable access to Google Cloud services such as BigQuery and Gemini.

- Check that the BigQuery API is enabled in your project by visiting https://console.cloud.google.com/apis/library/bigquery.googleapis.com.

### Authentication

Create a **service account** at https://console.cloud.google.com/iam-admin/serviceaccounts/create. A service account is used by an application to access Google Cloud Platform's services and has an associated email address (different from your own).


- Give the account an appropriate name, and under step 2 (Grant this service account access to project (optional)), choose "Owner" under the "Select a Role" dropdown.

- Ignore step 3 and click "Done".

Go to https://console.cloud.google.com/iam-admin/serviceaccounts to create a **service account key**. This will be downloaded to your computer so that you can connect to the BigQuery API via this Jupyter notebook.

- Select your recently created project.
- Click the email address of the service account.
- Click the Keys tab.
- Click the Add key drop-down menu, then select Create new key.
- Select JSON as the Key type and click Create.
- The keys will get saved to your computer.

Note the location and copy the file path (of the json file) to somewhere safe, for future reference.

See here for more information:

Service Account creation: https://cloud.google.com/iam/docs/service-accounts-create#creating (under Console)

Service Account key creation: https://cloud.google.com/iam/docs/keys-create-delete#iam-service-account-keys-create-console (under Console)


### Using the Python API

Google provides Python libraries for wrapping the Google APIs.

Installing the "google-cloud-bigquery", "google-cloud-storage", and "google-cloud-bigquery-storage" libraries should cover all the dependencies for the BigQuery section of this lab.

In [1]:
'''
!pip install google-cloud-bigquery
!pip install google-cloud-storage
!pip install google-cloud-bigquery-storage 
!pip install google-cloud-bigquery[pandas]
'''


'\n!pip install google-cloud-bigquery\n!pip install google-cloud-storage\n!pip install google-cloud-bigquery-storage \n!pip install google-cloud-bigquery[pandas]\n'

!pip install google-cloud-bigquery-storage # has additional capabilities for reading data from BigQuery using the BigQuery Storage API

In [2]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import bigquery_storage

Invoke a method of the `.Client` object that takes the path to your key files as a string argument:

In [3]:
key_path = r'resolute-mote-462623-v5-c4c05ed61421.json'          #: Change this to match your key filename

This should not throw an error if key retrieval / assignment worked:

In [4]:
storage_client = storage.Client.from_service_account_json(key_path)

*Nb. The `storage` object was used in the above example, but there are other objects of interest that have polymorphic `Client` members that are used similarly, such as `bigquery`, which is used below.*

Next, execute this:

In [5]:
client = bigquery.Client.from_service_account_json(key_path)

This client is associated with the default project (which was set or defaulted in the BigQuery UI):

In [6]:
client.project

'resolute-mote-462623-v5'

A BigQuery project contains datasets. Datasets contain tables. To get at the data in a table we need to create a reference that covers this hierarchy; in the `bigquery` library this looks like `project.dataset.table`.  

(Nb. Queries can be performed on projects and datasets, but most queries are performed on tables.)

To explore the public datasets we will start by reassigning our `client` variable using optional `project` parameter (set to `bigquery-public-data`):

In [7]:
#project = 'bigquery-public-data'
client = bigquery.Client.from_service_account_json(key_path, project = 'bigquery-public-data')
print(client.project)

bigquery-public-data


Here is how to get a list of the datasets in the current project:

In [8]:
datasets = list(client.list_datasets())
print(datasets)

[<google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492E37BF0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492DB6600>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023491ECE930>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EFB290>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EF85F0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EFA630>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EF96A0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EF93A0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EFB110>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492EFBA10>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492F41AF0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000023492F41B50>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000

That wasn't helpful. We need to go deeper into the object structure to get at something meaningful. Below is a function that exploits the `format` method of `project` and `dataset_id`, providing an easy way to list datasets:

In [9]:
# function for listing datasets in a project:
def printDatasetList(client):
    project = client.project    #: only one project can be associated with a client instance
    datasets = list(client.list_datasets())
    if datasets:
        print('Datasets in project {}:'.format(project))
        for dataset in datasets:
            print('\t{}'.format(dataset.dataset_id))
        found = True
    else:
        print('{} project does not contain any datasets.'.format(project))
        found = False
    return found

In [10]:
# list datasets in the default project:
flag = printDatasetList(client)  #: assigning to `flag` suppresses printing the return value (normally `True`)

Datasets in project bigquery-public-data:
	america_health_rankings
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bbc_news
	bigqueryml_ncaa
	bitcoin_blockchain
	blackhole_database
	blockchain_analytics_ethereum_mainnet_us
	bls
	bls_qcew
	breathe
	broadstreet_adi
	catalonian_mobile_coverage
	catalonian_mobile_coverage_eu
	census_bureau_acs
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	census_opportunity_atlas
	census_utility
	cfpb_complaints
	chicago_crime
	chicago_taxi_trips
	clemson_dice
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	cms_synthetic_patient_data_omop
	country_codes
	covid19_aha
	covid19_covidtracking
	covid19_ecdc
	covid19_ecdc_eu
	covid19_genome_sequence
	covid19_geotab_mobility_impact
	covid19_geotab_mobility_impact_eu
	covid19_google_mobility
	covid19_google_mobility_eu
	covid19_govt_response
	covid19_italy
	covid19_italy_eu
	covid19_jhu_csse
	covid19_jhu_csse_eu
	covid19_nyt
	covid19_open_dat

This list should correspond to what is shown here https://bigquery.cloud.google.com/publicdatasets under the **bigquery-public-data** item.

Here is how to create a dataset reference object by assigning a project and a dataset name:

In [11]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id)

If our current project was something other than `bigquery-public-data`, we could still create this reference by specifying the project that contains the dataset:

In [12]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id, project = 'bigquery-public-data')

How can we get the path of the dataset?

In [13]:
#ANSWER:
print(dataset_ref.path)

/projects/bigquery-public-data/datasets/samples


Explore more of this object's members:

*(HINT: You can type `dataset_ref.` in a new line, then hit the [Tab] key to see the available members for the object.)*

In [14]:
#?
#dataset_ref.

Here is a function for listing the tables in a dataset:

In [15]:
# function for listing tables in a dataset:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project = project)
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset {}:'.format(dataset_id))
        for table in tables:
            print('\t{}'.format(table.table_id))
        found = True
    else:
        print('{} dataset does not contain any tables.'.format(dataset_id))
        found = False
    return found

Use this function to list the tables in the current dataset:

In [16]:
#ANSWER
printTableList(client, dataset_id)

Tables in dataset samples:
	github_nested
	github_timeline
	gsod
	natality
	shakespeare
	trigrams
	wikipedia


True

To create a reference to a table within the dataset, we use the `table_id` attribute:

In [17]:
table_id = 'shakespeare'
table_ref = dataset_ref.table(table_id)

To access the data in the table itself, we use the `get_table()` method:

In [18]:
table = client.get_table(table_ref)  # API Request

NOTE: The contents of the table are not actually in our memory after this call! We are working with a Big Data platform, now, and we could easily end up pulling GBs or TBs of data by accident.

To minimise data bandwidth, memory consumption, and processing time, Big Data platforms employ ***lazy evaluation***. This means that no computation or data transfer actually takes place until we *realise* (use) the data. Even if we execute subsequent code that performs calculations on the data, no data flow or computation actually occurs until we request output (e.g. by executing a print to stdout or writing to a file).

What kind of object is returned by `client.get_table`?

In [19]:
#ANSWER:
print(type(client.get_table(table_ref)))

<class 'google.cloud.bigquery.table.Table'>


How can we view the design of the table (column names and types)? The name of the object attribute we need is the same term we learned in the module on databases:

In [20]:
#ANSWER
print(client.get_table(table_ref).schema)

[SchemaField('word', 'STRING', 'REQUIRED', None, 'A single unique word (where whitespace is the delimiter) extracted from a corpus.', (), None), SchemaField('word_count', 'INTEGER', 'REQUIRED', None, 'The number of times this word appears in this corpus.', (), None), SchemaField('corpus', 'STRING', 'REQUIRED', None, 'The work from which this word was extracted.', (), None), SchemaField('corpus_date', 'INTEGER', 'REQUIRED', None, 'The year in which this corpus was published.', (), None)]


Again, this is messy. If we wanted to refer to the column names and types in code, we might use something like this (which we could then parse into a dict):

In [21]:
result = ["{0} {1}".format(schema.name,schema.field_type) for schema in table.schema]
print(result)

['word STRING', 'word_count INTEGER', 'corpus STRING', 'corpus_date INTEGER']


But if we just want to print them, here is another neat function for that:

In [22]:
# function to print a table schema:
def printTableSchema(aTable):
    schemas = list(aTable.schema)
    if schemas:
        print('Table schema for {}:'.format(aTable.table_id))
        for aSchema in schemas:
            print('\t{0} {1}'.format(aSchema.name, aSchema.field_type))
        found = True
    else:
        found = False
    return found

Use this function to print the table schema:

In [23]:
#ANSWER:
printTableSchema(client.get_table(table_ref))

Table schema for shakespeare:
	word STRING
	word_count INTEGER
	corpus STRING
	corpus_date INTEGER


True

Now that we know what the columns are, we can write queries. Actually, we construct a query job by assigning an SQL statement to a method of the `client` object:

In [24]:
sql = "SELECT COUNT(1) FROM bigquery-public-data.samples.shakespeare"
'''
# MEANT TO FAIL

query_job = client.query(sql)
'''

'\n# MEANT TO FAIL\n\nquery_job = client.query(sql)\n'

This will throw an error since we don't have permission to create queries inside the `bigquery-public-data` project. Instead we set the project to our BigQuery project name.

In [25]:
client = bigquery.Client.from_service_account_json(key_path, project = 'resolute-mote-462623-v5') #<<< your BigQuery project ID here!
query_job = client.query(sql)

If that worked, show what query_job is:

In [26]:
# ANSWER
print(query_job)

QueryJob<project=resolute-mote-462623-v5, location=US, id=e86415f1-ab62-48ad-bcc7-6eafb2ab78c9>


Once again, due to lazy execution, no actual execution occurs until we request output:

In [27]:
for row in query_job:  # API request - fetches results
    print(row)

Row((164656,), {'f0_': 0})


And, again, we need to manipulate this to make it neat. Each member of the rowset is a list and we only want to extract the value, which is in the first member:

In [28]:
print(row[0])

164656


So, we now know that this table has 164,656 rows. (We would not want to print it!)

Write, execute, and print the results of a query that fetches 10 rows from the table, each containing the "word", "word_count", and "corpus" fields:

In [29]:
#ANSWER
sql = "SELECT word,word_count,corpus FROM bigquery-public-data.samples.shakespeare LIMIT 10"
client = bigquery.Client.from_service_account_json(key_path, project = 'resolute-mote-462623-v5') #<<< your BigQuery project ID here!
query_job = client.query(sql)

Whenever you catch yourself writing a swag of code to do something that seems rudimentary or low-level, there is a very good chance that you don't need to. A much easier way to handle the above requirement is to use the `to_dataframe` method of the QueryJob object:

In [30]:
df = query_job.to_dataframe()
df

,word,word_count,corpus
0,LVII,1,sonnets
1,augurs,1,sonnets
2,dimm'd,1,sonnets
3,plagues,1,sonnets
4,treason,1,sonnets
5,surmise,1,sonnets
6,heed,1,sonnets
7,Unthrifty,1,sonnets
8,quality,1,sonnets
9,wherever,1,sonnets


#### Additional Notes

1. Here is a readable way to code long SQL statements:

In [31]:
sql = """
    SELECT word, word_count, corpus
    FROM bigquery-public-data.samples.shakespeare
    LIMIT 10
    """

2. If you had an application that needed to modify the tables or datasets in the `bigquery-public-data` is project, you could copy them to our own project, where you would have the permissions to do as you please with the data (subject to Google's terms of use).

3. We aren't limited to the datasets that are already in BigQuery. We can upload tables from our computer, and we can pull data in from other online sources.

## Google Gemini

Google Gemini (formerly Bard) is a multimodal generative AI chatbot. It can process text, audio, images and video.

Create an API key at https://aistudio.google.com/app/apikey . Copy the key and paste it into a text file called 'gemini_key.txt'

## Google Gemini UI
While signed into Google experiment with some prompts at https://aistudio.google.com/app/prompts/new_chat. A prompt gallery is available at https://aistudio.google.com/app/gallery.

## Google Gemini API

The library `google-generativeai` gives access to Gemini models. For this section download the following two files from the DATA folder:

* `equation.jpg`
* `JFK.mp3`

In [32]:
# -U gives the latest version
!pip install -U google-genai

In [33]:
from google import genai
from IPython.display import Markdown # allows Markdown text to be displayed in the notebook

Firstly we read our API key from `gemini_key.txt`:

In [34]:
filename = 'gemini_key.txt'
try:
    with open(filename, 'r') as f:
        # It's assumed your file contains a single line containing your API key only
        key = f.read().strip()
except FileNotFoundError:
    print("'%s' file not found" % filename)

In [35]:
client = genai.Client(api_key=key)

We shall use the following client methods:

* `models.generate_content()`: used to generate responses from the model
* `models.list()`: used to see available models
* `files.upload()`: used to upload image/audio files

The following code lists the available models for text generation:

In [36]:
for m in client.models.list():
    if "generateContent" in m.supported_actions:
        print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lit

As suggested by the name, Gemini Flash is designed for faster responses while Gemini Pro works better at more challenging tasks. Pro has a lower rate limit of 2 requests per minute as seen in https://ai.google.dev/pricing.

In [37]:
response = client.models.generate_content(
    model="gemini-1.5-flash", contents="What is an API?"
)

The attribute `text` shows in Markdown format the response of the model to the question.

In [38]:
Markdown(response.text)

API stands for **Application Programming Interface**.  It's essentially a messenger that takes requests from one application and tells another application what to do.  Think of it as a menu in a restaurant.  You (the application) don't need to know how the kitchen (the other application) prepares your food, you just need to know what options are available (the API's functions) and how to order them (the API's requests).

More technically, an API is a set of rules and specifications that software programs can follow to communicate with each other.  It defines how software components should interact, what data they can exchange, and the format of that data.  This allows different software systems, built by different developers, to work together seamlessly.

Here's a breakdown:

* **Requests:**  An application makes a request to the API, specifying what it needs.
* **Response:** The API processes the request and sends back a response, which may include data or a confirmation message.
* **Data formats:** APIs commonly use formats like JSON or XML to exchange data.
* **Endpoints:** Specific URLs that applications use to access the API's functions.

**Examples:**

*  When you use a weather app on your phone, the app uses an API to fetch weather data from a weather service.
*  A social media login button on a website uses an API to authenticate users with their social media accounts.
*  A mapping application uses APIs to display maps and directions.

In short, APIs are crucial for building modern software applications because they enable modularity, reusability, and integration between different systems.


Study the response object and identify the total token count.

In [42]:
# ANSWER
response.usage_metadata.total_token_count

344

Next, we have Gemini process a mathematical equation in an image.

In [43]:
sample_image = client.files.upload(file=r"C:\Users\koh_k\Downloads\Institute of Data\DATA\equation.jpg")

In [44]:
result = client.models.generate_content(
    model="gemini-1.5-flash",
    contents=[
        sample_image,
        "\n\n",
        "What is in this image?",
    ],
)
Markdown(result.text)

The image contains a quadratic equation:

0 = x² - 5x + 6

Use the Gemini 1.5 Flash model to solve the equation in the image.

In [49]:
# REPLACE ??? with code
'''
response = model.generate_content([prompt, sample_image])
Markdown(">" + response.text)
'''

result = client.models.generate_content(
    model="gemini-1.5-flash",
    contents=[
        sample_image,
        "\n\n",
        "Solve the quadratic equation in this image",
    ],
)

Markdown(">" + result.text)

>Here's how to solve the quadratic equation 0 = x² - 5x + 6:

**1. Factor the quadratic expression:**

We need to find two numbers that add up to -5 (the coefficient of x) and multiply to 6 (the constant term). Those numbers are -2 and -3.  Therefore, the factored form is:

0 = (x - 2)(x - 3)

**2. Set each factor to zero and solve:**

* x - 2 = 0  =>  x = 2
* x - 3 = 0  =>  x = 3

**Solution:**

The solutions to the quadratic equation are x = 2 and x = 3.

Finally we transcribe a short audio clip.

In [51]:
# REPLACE ??? with code
audio_file = client.files.upload(file=r"C:\Users\koh_k\Downloads\Institute of Data\DATA\JFK.mp3")

# Create a prompt.
prompt = "Transcribe the audio."

# Pass the prompt and the audio file to gemini-1.5-flash using client.models.generate_content
response = client.models.generate_content(
    model="gemini-1.5-flash",
    contents=[
        audio_file,
        "\n\n",
        "Transcribe the audio file"
    ]
)

# Print the response.
print(response.text)

And so my fellow Americans, ask not what your country can do for you, ask what you can do for your country.



## Further reading

If you wish to pick up a few more skills in BigQuery you can go to https://cloud.google.com/bigquery/create-simple-app-api and https://cloud.google.com/bigquery/docs/samples.

Alternatively, you can take a deeper dive into the API here: https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html.

The Google Gemini API documentation is at https://ai.google.dev/gemini-api/docs

## - END -



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



